In [255]:
GYRO = False
SEQUENCE_LENGTH = 5
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 30
CNN = True
MODEL_NAME = f"AllInOne_CNN:{CNN}_pochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 3
TEST_SIZE = 7
LEAVE_ONE_OUT = True
NORMALISE = True
OVERLAP_ON_TEST_SET = False
TRANSFORM = True
AUGMENT = False
AUGMENT_SIZE = 1 # 1 means 1 extra augmented sample per sample, 2 means 2 extra augmented samples per sample etc.
AUGMENT_NOISE = 0.004
DIFFERENCES = True
DERIVATIVES = True
RANDOM_TEST_SET = False

# only use this when doing leave one out
EARYLY_STOPPING = True



### This code takes in a position, eg "sitting_or_standing", "lying_down_left" etc and trains a model for just classifying activities of that position

In [256]:
import helpers.file_tagger as file_tagger
import helpers.sequence_generator as sequence_generator
import helpers.split_by_student as split_by_student
from keras.layers import LeakyReLU

import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models, regularizers
from sklearn.model_selection import train_test_split
import time
from keras.callbacks import EarlyStopping



POSSIBLE_ACTIVITIES = [
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",
    
    "lying_down_left&normal_breathing",
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",

    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",

    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",

    "lying_down_stomach&normal_breathing",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",

]
  

POSSIBLE_OUTCOMES= [
    "sitting_or_standing&normal_breathing",
    "sitting_or_standing&coughing",
    "sitting_or_standing&hyperventilating",

    "lying_down_left&normal_breathing",
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",

    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",

    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",

    "lying_down_stomach&normal_breathing",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",    
]

DATA_DIRECTORY = "./all_respeck"
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}

if OVERLAP_ON_TEST_SET:
    TEST_SEQUENCE_OVERLAP = SEQUENCE_OVERLAP
else:
    TEST_SEQUENCE_OVERLAP = 0

In [257]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in POSSIBLE_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if csv_file in file_names:
                if gyro:
                    sequences = sequence_generator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)
                else:
                    sequences = sequence_generator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)

                # iterate through each generated sequence
                for sequence in sequences:
                    position = key.split("&")[0]
                    activity = key.split("&")[1]

                    if activity == "talking" or activity == "singing" or activity == "laughing" or activity == "eating":
                        activity = "other"

                    if position == "standing" or position == "sitting":
                        position = "sitting_or_standing"
                        
                    tagged_data.append((position + "&" + activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")

    return tagged_data

In [258]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    print("Training CNN model")
    
    width = 3
    if TRANSFORM:
        width = width+3
    if DIFFERENCES:
        width = width+3
    if DERIVATIVES:
        width = width+3
    if GYRO:
        width = width *2
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),


        
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),

        layers.Dropout(0.2),

        
        layers.Conv1D(128, 3, activation='relu'), 
        layers.MaxPooling1D(2),

        layers.Dropout(0.5),
       
        layers.Flatten(),
        
        layers.Dense(128, activation='relu', activity_regularizer=regularizers.l2(0.01)),
        layers.Dense(64, activation='relu'), 
        #layers.Dense(64, activation='relu'),
        #layers.Dense(32, activation='relu'),
        #layers.Dense(64, activation='relu', activity_regularizer=regularizers.l2(0.0001)), 
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if not EARYLY_STOPPING:
        if len(validation_data[0]) == 0:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
        else:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)
    else:
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max', restore_best_weights=True)
        if len(validation_data[0]) == 0:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])
        else:
            model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data, callbacks=[early_stopping])


    return model

In [259]:
def train_model_LSTM(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    print("\n\n ===== Training LSTM Model =====\n")
    if GYRO:
        width = 6
    else:
        width = 3
    if TRANSFORM:
        width = width+3
    if DIFFERENCES:
        width = width+3
    model = Sequential([
        layers.LSTM(12, activation='relu', return_sequences=True, input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.BatchNormalization(),
        
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [260]:
def augment_data(sequences, labels, num_augmentations):
    augmented_sequences = []
    augmented_labels = []
    
    for sequence, label in zip(sequences, labels):
        for _ in range(num_augmentations):
            augmented_sequence = sequence + np.random.uniform(-AUGMENT_NOISE, AUGMENT_NOISE, size=sequence.shape)
            augmented_sequences.append(augmented_sequence)
            augmented_labels.append(label)
    
    return augmented_sequences, augmented_labels


In [261]:
if not GYRO:
    def fft(data):

        # Extract x, y, and z data
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]

        # Apply FFT to each axis
        x_fft = np.fft.fft(x_data)
        y_fft = np.fft.fft(y_data)
        z_fft = np.fft.fft(z_data)

        # The result is complex numbers, so you may want to take the magnitude
        x_magnitude = np.abs(x_fft)
        y_magnitude = np.abs(y_fft)
        z_magnitude = np.abs(z_fft)

        representation = []
        for i in range(len(x_magnitude)):
            representation.append([x_magnitude[i], y_magnitude[i], z_magnitude[i]]) #, x_frequencies[i], y_frequencies[i], z_frequencies[i]])

        return representation

    def extract_fft(train_data, dev_data, test_data):
        train_features = [fft(sequence) for sequence in train_data]
        dev_features = [fft(sequence) for sequence in dev_data]
        test_features = [fft(sequence) for sequence in test_data]

        return train_features, dev_features, test_features


    def merge_arrays(arr1, arr2):
        return np.concatenate((arr1, arr2), axis=1)

    def normalise(sequence):
        """
        Normalizes a matrix of accelerometer values.
        """
        norm = np.linalg.norm(sequence, axis=1)
        norm[norm == 0] = 1
        return sequence / norm[:, np.newaxis]

    def normalise_data(data):
        """
        Normalizes a list of matrices of accelerometer values.
        """
        return [normalise(sequence) for sequence in data]

    def differential(data):
        # Extract x, y, and z data
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]

        # Compute the differences between consecutive data points
        x_diff = np.diff(x_data)
        y_diff = np.diff(y_data)
        z_diff = np.diff(z_data)

        # Add a 0 at the start of the differential variables
        x_diff = np.insert(x_diff, 0, 0)
        y_diff = np.insert(y_diff, 0, 0)
        z_diff = np.insert(z_diff, 0, 0)
        
        # Combine the differential values into a representation
        representation = []
        for i in range(len(x_diff)):
            representation.append([x_diff[i], y_diff[i], z_diff[i]])

        return representation

    def extract_differentials(train_data, dev_data, test_data):
        train_features = [differential(sequence) for sequence in train_data]
        
        dev_features = [differential(sequence) for sequence in dev_data]
        
        test_features = [differential(sequence) for sequence in test_data]

        return train_features, dev_features, test_features

    def derivative(data):
        # Extract x, y, and z data
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]

        # Compute the derivative of the data
        x_derivative = np.gradient(x_data)
        y_derivative = np.gradient(y_data)
        z_derivative = np.gradient(z_data)

        # Combine the derivative values into a representation
        representation = []
        for i in range(len(x_derivative)):
            representation.append([x_derivative[i], y_derivative[i], z_derivative[i]])

        return representation

    def extract_gradients(train_data, dev_data, test_data):
        train_features = [derivative(sequence) for sequence in train_data]
        dev_features = [derivative(sequence) for sequence in dev_data]
        test_features = [derivative(sequence) for sequence in test_data]

        return train_features, dev_features, test_features

In [262]:
if GYRO:
    def fft(data):

        # Extract x, y, and z data
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        x_gyro = data[:, 3]
        y_gyro = data[:, 4]
        z_gyro = data[:, 5]

        # Apply FFT to each axis
        x_fft = np.fft.fft(x_data)
        y_fft = np.fft.fft(y_data)
        z_fft = np.fft.fft(z_data)
        x_gyro_fft = np.fft.fft(x_gyro)
        y_gyro_fft = np.fft.fft(y_gyro)
        z_gyro_fft = np.fft.fft(z_gyro)

        # The result is complex numbers, so you may want to take the magnitude
        x_magnitude = np.abs(x_fft)
        y_magnitude = np.abs(y_fft)
        z_magnitude = np.abs(z_fft)
        x_gyro_magnitude = np.abs(x_gyro_fft)
        y_gyro_magnitude = np.abs(y_gyro_fft)
        z_gyro_magnitude = np.abs(z_gyro_fft)

        representation = []
        for i in range(len(x_magnitude)):
            representation.append([x_magnitude[i], y_magnitude[i], z_magnitude[i], x_gyro_magnitude[i], y_gyro_magnitude[i], z_gyro_magnitude[i]]) #, x_frequencies[i], y_frequencies[i], z_frequencies[i]])

        return representation

    def extract_fft(train_data, dev_data, test_data):
        train_features = [fft(sequence) for sequence in train_data]
        dev_features = [fft(sequence) for sequence in dev_data]
        test_features = [fft(sequence) for sequence in test_data]

        return train_features, dev_features, test_features


    def merge_arrays(arr1, arr2):
        return np.concatenate((arr1, arr2), axis=1)

    def normalise(sequence):
        """
        Normalizes a matrix of accelerometer values.
        """
        norm = np.linalg.norm(sequence, axis=1)
        norm[norm == 0] = 1
        return sequence / norm[:, np.newaxis]

    def normalise_data(data):
        """
        Normalizes a list of matrices of accelerometer values.
        """
        return [normalise(sequence) for sequence in data]

    def differential(data):
        # Extract x, y, and z data
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        x_gyro = data[:, 3]
        y_gyro = data[:, 4]
        z_gyro = data[:, 5]

        # Compute the differences between consecutive data points
        x_diff = np.diff(x_data)
        y_diff = np.diff(y_data)
        z_diff = np.diff(z_data)
        x_gyro_diff = np.diff(x_gyro)
        y_gyro_diff = np.diff(y_gyro)
        z_gyro_diff = np.diff(z_gyro)

        # Add a 0 at the start of the differential variables
        x_diff = np.insert(x_diff, 0, 0)
        y_diff = np.insert(y_diff, 0, 0)
        z_diff = np.insert(z_diff, 0, 0)
        x_gyro_diff = np.insert(x_gyro_diff, 0, 0)
        y_gyro_diff = np.insert(y_gyro_diff, 0, 0)
        z_gyro_diff = np.insert(z_gyro_diff, 0, 0)
        
        # Combine the differential values into a representation
        representation = []
        for i in range(len(x_diff)):
            representation.append([x_diff[i], y_diff[i], z_diff[i], x_gyro_diff[i], y_gyro_diff[i], z_gyro_diff[i]])

        return representation

    def extract_differentials(train_data, dev_data, test_data):
        train_features = [differential(sequence) for sequence in train_data]
        
        dev_features = [differential(sequence) for sequence in dev_data]
        
        test_features = [differential(sequence) for sequence in test_data]

        return train_features, dev_features, test_features

    def derivative(data):
        # Extract x, y, and z data
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        x_gyro = data[:, 3]
        y_gyro = data[:, 4]
        z_gyro = data[:, 5]

        # Compute the derivative of the data
        x_derivative = np.gradient(x_data)
        y_derivative = np.gradient(y_data)
        z_derivative = np.gradient(z_data)
        x_gyro_derivative = np.gradient(x_gyro)
        y_gyro_derivative = np.gradient(y_gyro)
        z_gyro_derivative = np.gradient(z_gyro)

        # Combine the derivative values into a representation
        representation = []
        for i in range(len(x_derivative)):
            representation.append([x_derivative[i], y_derivative[i], z_derivative[i], x_gyro_derivative[i], y_gyro_derivative[i], z_gyro_derivative[i]])

        return representation

    def extract_gradients(train_data, dev_data, test_data):
        train_features = [derivative(sequence) for sequence in train_data]
        dev_features = [derivative(sequence) for sequence in dev_data]
        test_features = [derivative(sequence) for sequence in test_data]

        return train_features, dev_features, test_features

In [263]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size, randomize=RANDOM_TEST_SET)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [264]:
def leave_one_out(isCNN=CNN):
    students = split_by_student.get_list_of_stutents()
    test_accuracies = []
    for test_student in students:
        print("testing student: " + str(test_student))
        print("training students: " + str([student for student in students if student != test_student]))
        
        test_files, training_files = split_by_student.get_list_of_files(test_student)

        tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
        tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_SEQUENCE_OVERLAP, file_names=test_files)

        train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
        test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

        if AUGMENT:
            augmented_train_data, augmented_train_labels = augment_data(train_data, train_labels, AUGMENT_SIZE)
            train_data = np.concatenate((train_data, augmented_train_data))
            train_labels = np.concatenate((train_labels, augmented_train_labels))
            print("length of training data with augmentation: " + str(len(train_data)))

        if TRANSFORM:
            train_transform, dev_transform, test_transform = extract_fft(train_data, [], test_data)
            train_data = np.array(train_data)
            train_transform = np.array(train_transform)
           
            test_data = np.array(test_data)
            test_transform = np.array(test_transform)

            train_data = np.array([merge_arrays(train_data[i], train_transform[i]) for i in range(len(train_data))])
            test_data = np.array([merge_arrays(test_data[i], test_transform[i]) for i in range(len(test_data))])

        if DIFFERENCES:
            train_differential, dev_differential, test_differential = extract_differentials(train_data, [], test_data)
            train_data = np.array(train_data)
            train_differential = np.array(train_differential)
           
            test_data = np.array(test_data)
            test_differential = np.array(test_differential)

            train_data = np.array([merge_arrays(train_data[i], train_differential[i]) for i in range(len(train_data))])
            test_data = np.array([merge_arrays(test_data[i], test_differential[i]) for i in range(len(test_data))])

        
        if DERIVATIVES:
            train_derivatives, dev_derivatives, test_derivatives = extract_gradients(train_data, [], test_data)
            train_data = np.array(train_data)
            train_derivatives = np.array(train_derivatives)
         
            test_data = np.array(test_data)
            test_derivatives = np.array(test_derivatives)

            train_data = np.array([merge_arrays(train_data[i], train_derivatives[i]) for i in range(len(train_data))])
            test_data = np.array([merge_arrays(test_data[i], test_derivatives[i]) for i in range(len(test_data))])


        if isCNN:
            model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels))
        else:
            model = train_model_LSTM(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels))
        
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        test_accuracies.append(test_accuracy)
        print("for student " + str(test_student) + " the accuracy is " + str(test_accuracy))
        print("average accuracy so far: " + str(sum(test_accuracies)/len(test_accuracies)))
        print("number of students tested so far: " + str(len(test_accuracies)))
        time.sleep(3)

        
    print("Accuracy for each student:")
    print(", ".join([f"{student}: {accuracy}" for student, accuracy in zip(students, test_accuracies)]))
    print("Average overall accuracy:", sum(test_accuracies)/len(test_accuracies))


In [265]:
if LEAVE_ONE_OUT:
    leave_one_out()
    exit()

if not LEAVE_ONE_OUT:

    train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)

    if AUGMENT:
        augmented_train_data, augmented_train_labels = augment_data(train_data, train_labels, AUGMENT_SIZE)
        train_data = np.concatenate((train_data, augmented_train_data))
        train_labels = np.concatenate((train_labels, augmented_train_labels))
        print("length of training data with augmentation: " + str(len(train_data)))

    if TRANSFORM:
        train_transform, dev_transform, test_transform = extract_fft(train_data, dev_data, test_data)
        train_data = np.array(train_data)
        train_transform = np.array(train_transform)
        dev_data = np.array(dev_data)
        dev_transform = np.array(dev_transform)
        test_data = np.array(test_data)
        test_transform = np.array(test_transform)

        train_data = np.array([merge_arrays(train_data[i], train_transform[i]) for i in range(len(train_data))])
        dev_data = np.array([merge_arrays(dev_data[i], dev_transform[i]) for i in range(len(dev_data))])
        test_data = np.array([merge_arrays(test_data[i], test_transform[i]) for i in range(len(test_data))])

    if DIFFERENCES:
        train_differential, dev_differential, test_differential = extract_differentials(train_data, dev_data, test_data)
        train_data = np.array(train_data)
        train_differential = np.array(train_differential)
        dev_data = np.array(dev_data)
        dev_differential = np.array(dev_differential)
        test_data = np.array(test_data)
        test_differential = np.array(test_differential)

        train_data = np.array([merge_arrays(train_data[i], train_differential[i]) for i in range(len(train_data))])
        dev_data = np.array([merge_arrays(dev_data[i], dev_differential[i]) for i in range(len(dev_data))])
        test_data = np.array([merge_arrays(test_data[i], test_differential[i]) for i in range(len(test_data))])

    if DERIVATIVES:
        train_derivatives, dev_derivatives, test_derivatives = extract_gradients(train_data, dev_data, test_data)
        train_data = np.array(train_data)
        train_derivatives = np.array(train_derivatives)
        dev_data = np.array(dev_data)
        dev_derivatives = np.array(dev_derivatives)
        test_data = np.array(test_data)
        test_derivatives = np.array(test_derivatives)

        train_data = np.array([merge_arrays(train_data[i], train_derivatives[i]) for i in range(len(train_data))])
        dev_data = np.array([merge_arrays(dev_data[i], dev_derivatives[i]) for i in range(len(dev_data))])
        test_data = np.array([merge_arrays(test_data[i], test_derivatives[i]) for i in range(len(test_data))])

    if CNN:
        model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) 
    else:
        model = train_model_LSTM(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) 


    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_data, test_labels)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

testing student: s66
training students: ['s65', 's48', 's54', 's92', 's55', 's23', 's13', 's64', 's5', 's57', 's44', 's32', 's93', 's12', 's51', 's95', 's84', 's75', 's36', 's40', 's79', 's83', 's60', 's21', 's56', 's22', 's34', 's61', 's70', 's97', 's102', 's59', 's77', 's50', 's87', 's43', 's29', 's52', 's11', 's15', 's91', 's33', 's80', 's18', 's17', 's38', 's67', 's46', 's27', 's9', 's72', 's39', 's100', 's3', 's98', 's8', 's7', 's30', 's45', 's88', 's1', 's63', 's16', 's86', 's42', 's82', 's74', 's71', 's96', 's35']
there are 16218 tagged sequences in the dataset
there are 108 tagged sequences in the dataset
Training CNN model
Epoch 1/30
1622/1622 [==============================] - 7s 4ms/step - loss: 0.9230 - accuracy: 0.6955 - val_loss: 0.3509 - val_accuracy: 0.8981
Epoch 2/30
1622/1622 [==============================] - 6s 4ms/step - loss: 0.4959 - accuracy: 0.8362 - val_loss: 0.2873 - val_accuracy: 0.8981
Epoch 3/30
1622/1622 [==============================] - 6s 3ms/step - lo

KeyboardInterrupt: 

In [ ]:
# Save the trained model
#model.save(f"models/presentation_models/{MODEL_NAME}_GYRO_{GYRO}_{test_accuracy}.keras")

# MINIMIZING TRAINING LOSS:
# currently 0.9906 with conv: 64, 64, 128 and dense: 128 64
# Now attempting regularization with l2

# Testing addition of GYRO (same test set is used for both as RANDOM_TEST_SET is false) (647 in dev, 647 in test)
# score without gyro: 0.8578
# score with gyro: 0.8902

# highest dev number so far is 0.87
